# **Decision Trees**

The Wisconsin Breast Cancer Dataset(WBCD) can be found here(https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data)

This dataset describes the characteristics of the cell nuclei of various patients with and without breast cancer. The task is to classify a decision tree to predict if a patient has a benign or a malignant tumour based on these features.

Attribute Information:
```
#  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)
```

In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import random

In [58]:
headers = ["ID", "CT", "UCSize", "UCShape", "MA",\
           "SECSize", "BN", "BC", "NN", "Mitoses", "Diagnosis"]
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data', 
                   na_values='?', 
                   header=None, 
                   index_col=['ID'], 
                   names = headers) 
data = data.reset_index(drop=True)
data = data.fillna(0)
data.describe()

,CT,UCSize,UCShape,MA,SECSize,BN,BC,NN,Mitoses,Diagnosis
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.463519,3.437768,2.866953,1.589413,2.689557
std,2.815741,3.051459,2.971913,2.855379,2.214300,3.640708,2.438364,3.053634,1.715078,0.951273
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000,4.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [59]:
# Show first 10 columns of data
data.head(10)

,CT,UCSize,UCShape,MA,SECSize,BN,BC,NN,Mitoses,Diagnosis
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2
5,8,10,10,8,7,10.0,9,7,1,4
6,1,1,1,1,2,10.0,3,1,1,2
7,2,1,2,1,2,1.0,3,1,1,2
8,2,1,1,1,2,1.0,1,1,5,2
9,4,2,1,1,2,1.0,2,1,1,2


In [60]:
#Checking for empty cells.
data.isnull().sum()

CT           0
UCSize       0
UCShape      0
MA           0
SECSize      0
BN           0
BC           0
NN           0
Mitoses      0
Diagnosis    0
dtype: int64

In [61]:
# Define Features and labels i.e., X-data and Y-data
label = data['Diagnosis'].values
Features = data.drop('Diagnosis', axis=1)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(Features, label, test_size=0.33, random_state=42)

1. a) Implement a decision tree (you can use decision tree implementation from existing libraries).

   b) Train a decision tree object of the above class on the WBC dataset using **misclassification rate**, **entropy** and **Gini** as the splitting metrics.

   c) Report the accuracies in each of the above splitting metrics and give the best result. 

$\LARGE 1a)$

In [75]:
# We've defined noly for Misclassification rate.

def misclassification_rate(prob):
    misc_rate = 1 - max(prob, 1-prob)

    return misc_rate

def misclassification_score(left_child_node, right_child_node):    
    parent = left_child_node + right_child_node

    prob_left = left_child_node.count(1) / len(left_child_node) if len(left_child_node) > 0 else 0
    prob_right = right_child_node.count(1) / len(right_child_node) if len(right_child_node) > 0 else 0

    misc_score_left = misclassification_rate(prob_left)
    misc_score_right = misclassification_rate(prob_right)

    return (len(left_child_node)/len(parent))*misc_score_left + (len(right_child_node)/len(parent))*misc_score_right

def find_split_point(X_bootstrap, y_bootstrap, max_features):
    """Given maximum no. of features at a node try to find the best split point according
    to impurity score for each threshold or possible split point."""
    feature_ls = list()
    num_features = len(X_bootstrap[0])
    
    while len(feature_ls) <= max_features:
        feature_idx = random.sample(range(num_features), 1)
        if feature_idx not in feature_ls:
            feature_ls.extend(feature_idx)
    
    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
        for split_point in X_bootstrap[:,feature_idx]:
            left_child_node = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child_node = {'X_bootstrap': [], 'y_bootstrap': []}
            
            # split children for continuous variables
            if type(split_point) in [int, float]:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value <= split_point:
                        left_child_node['X_bootstrap'].append(X_bootstrap[i])
                        left_child_node['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child_node['X_bootstrap'].append(X_bootstrap[i])
                        right_child_node['y_bootstrap'].append(y_bootstrap[i])
            # split children for categoric variables
            else:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value == split_point:
                        left_child_node['X_bootstrap'].append(X_bootstrap[i])
                        left_child_node['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child_node['X_bootstrap'].append(X_bootstrap[i])
                        right_child_node['y_bootstrap'].append(y_bootstrap[i])
            
            split_info_gain = misclassification_score(left_child_node['y_bootstrap'],\
                                                      right_child_node['y_bootstrap'])
            
            if split_info_gain > best_info_gain:
                best_info_gain = split_info_gain
                left_child_node['X_bootstrap'] = np.array(left_child_node['X_bootstrap'])
                right_child_node['X_bootstrap'] = np.array(right_child_node['X_bootstrap'])
                node = {'information_gain': split_info_gain, 
                        'left_child': left_child_node, 
                        'right_child': right_child_node, 
                        'split_point': split_point,
                        'feature_idx': feature_idx}
                
    
    return node

def terminal_node(node):
    """Based on max count of labels/class, assign the label to leaf node"""
    y_bootstrap = node['y_bootstrap']
    pred = max(y_bootstrap, key = y_bootstrap.count)
    return pred


def split_node(node, max_features, min_samples_split, max_depth, depth):
    """Based on Impurity score try to split the node 
    subject to constraints like max_features, min_samples_spit, max_depth and depth."""
    left_child = node['left_child']
    right_child = node['right_child']

    del(node['left_child'])
    del(node['right_child'])

    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return

    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node

    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = node['right_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
        split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1)
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = node['left_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)
        
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    """Build decision tree given boostrapped data along with its labels 
    and the criteria to split + some extra factors."""
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features)
    split_node(root_node, max_features, min_samples_split, max_depth, 1)
    return root_node        
        
def predict_tree(tree, x_test_sample):
    """Predict the output for a single sample with a given tree."""
    feature_idx = tree['feature_idx']

    if x_test_sample[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], x_test_sample)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], x_test_sample)
        else:
            return tree['right_split']
        
def predict_all_tree(tree, X_test):
    """Return prediction for all samples in X_test"""
    if type(X_test) != np.ndarray:
        X_test = X_test.values
    return np.array([predict_tree(tree, X_test[i]) for i in range(len(X_test))])

$\LARGE 1b)\ \ \&\ \ \LARGE 1c)$

In [63]:
Decision_Tree = DecisionTreeClassifier(criterion="gini")
Decision_Tree.fit(X_train,y_train)
pred = Decision_Tree.predict(X_test)
print("With Gini as splitting metrics, the accuracy score = ", accuracy_score(y_test,pred))

With Gini as splitting metrics, the accuracy score =  0.935064935064935


In [64]:
Decision_Tree = DecisionTreeClassifier(criterion="entropy")
Decision_Tree.fit(X_train,y_train)
pred = Decision_Tree.predict(X_test)
print("With Entropy as splitting metrics, the accuracy score = ", accuracy_score(y_test,pred))

With Entropy as splitting metrics, the accuracy score =  0.9393939393939394


In [79]:
model_dt = build_tree(X_train.values, y_train, max_features=3, max_depth=3, min_samples_split=2)

preds = predict_all_tree(model_dt, X_test)
print("With Misclassification rate as splitting metrics, the accuracy score = ", accuracy_score(y_test, preds))

With Misclassification rate as splitting metrics, the accuracy score =  0.8917748917748918


$\LARGE 1d)$

1. d) Experiment with different approaches to decide when to terminate the tree (number of layers, purity measure, etc). Report and give explanations for all approaches. 

In [41]:
decision_tree_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=7, min_impurity_decrease=0.0015)
decision_tree_entropy.fit(X_train, y_train)
entropy_preds = decision_tree_entropy.predict(X_test)
print('Accuracy for entropy splitting metric with max_depth=7 and min limit for decrease in impurity=0.0015, is', accuracy_score(y_test, entropy_preds), '\n')

decision_tree_gini = DecisionTreeClassifier(criterion='gini', max_depth=7, min_impurity_decrease=0.0015)
decision_tree_gini.fit(X_train, y_train)
gini_preds = decision_tree_gini.predict(X_test)
print('Accuracy for gini splitting metric with max_depth=7 and min limit for decrease in impurity=0.0015, is', accuracy_score(y_test, gini_preds), '\n')

decision_tree_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=9, max_features='sqrt')
decision_tree_entropy.fit(X_train, y_train)
entropy_preds = decision_tree_entropy.predict(X_test)
print('Accuracy for entropy splitting metric with max_depth=9 and max_features=sqrt is', accuracy_score(y_test, entropy_preds), '\n')

decision_tree_gini = DecisionTreeClassifier(criterion='gini', max_depth=9, max_features='sqrt')
decision_tree_gini.fit(X_train, y_train)
gini_preds = decision_tree_gini.predict(X_test)
print('Accuracy for gini splitting metric with max_depth=9 and max_features=sqrt is', accuracy_score(y_test, gini_preds), '\n')

decision_tree_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=9, max_features='log2')
decision_tree_entropy.fit(X_train, y_train)
entropy_preds = decision_tree_entropy.predict(X_test)
print('Accuracy for entropy splitting metric with max_depth=9 and max_features=log2 is', accuracy_score(y_test, entropy_preds), '\n')

decision_tree_gini = DecisionTreeClassifier(criterion='gini', max_depth=9, max_features='log2')
decision_tree_gini.fit(X_train, y_train)
gini_preds = decision_tree_gini.predict(X_test)
print('Accuracy for gini splitting metric with max_depth=9 and max_features=log2 is', accuracy_score(y_test, gini_preds), '\n')

Accuracy for entropy splitting metric with max_depth=7 and min limit for decrease in impurity=0.0015, is 0.935064935064935 

Accuracy for gini splitting metric with max_depth=7 and min limit for decrease in impurity=0.0015, is 0.9523809523809523 

Accuracy for entropy splitting metric with max_depth=9 and max_features=sqrt is 0.9307359307359307 

Accuracy for gini splitting metric with max_depth=9 and max_features=sqrt is 0.935064935064935 

Accuracy for entropy splitting metric with max_depth=9 and max_features=log2 is 0.9567099567099567 

Accuracy for gini splitting metric with max_depth=9 and max_features=log2 is 0.9523809523809523 



2. What is boosting, bagging and  stacking? Which class does random forests belong to and why?

**Answer:**

1. **Boosting**:- Boosting is an ensemble iterative technique where homogenous weak learners **learn sequentially in an adaptive manner**, i.e., it adjusts the weights of observations based on the last classification. In other words it aims at solving the problem by training **a set of weak learners in series** with each learner learning the errors from previous learner and try to correct them.

2. **Bagging**:- A method homogenous weak learners learns independently from each other as a small sample population and combines their prediction following some deterministic averaging process. Informally, it is an ensemble technique which aims at solving the problem by training a set of weak learners in **parallel and combines the output** from all the learners to predict the final output.

3. **Stacking**:- Stacking is an ensemble technique which aims at solving problem by training a set of different models (may be homogenous or heterogenous weak learners) at intermediate level in parallel, and then train a final model (also called **meta model**) which learns from these intermediate models' output to predict the final output.

*Random Forest use bagging technique. They build a bunch of decision trees as weak learners then combine their outputs (through voting) to generate the final predictions.*

3. Implement random forest algorithm using different decision trees . 

In [42]:
def entropy(p):
    """Shannon's entropy score."""
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return - (p * np.log2(p) + (1 - p) * np.log2(1-p))

def information_gain(left_child_node, right_child_node):
    """Difference of Impurity score between parent and child nodes."""
    parent = left_child_node + right_child_node
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child_node.count(1) / len(left_child_node) if len(left_child_node) > 0 else 0
    p_right = right_child_node.count(1) / len(right_child_node) if len(right_child_node) > 0 else 0
    IG_p = entropy(p_parent)
    IG_l = entropy(p_left)
    IG_r = entropy(p_right)
    return IG_p - len(left_child_node) / len(parent) * IG_l - len(right_child_node) / len(parent) * IG_r

def gini_impurity(p):
    """An impurity score based on probabilty of labels residing in a node."""
    return 1 - np.square(p)

def gini_score(left_child_node, right_child_node):
    parent = left_child_node + right_child_node
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child_node.count(1) / len(left_child_node) if len(left_child_node) > 0 else 0
    p_right = right_child_node.count(1) / len(right_child_node) if len(right_child_node) > 0 else 0
    gini_l = gini_impurity(p_left)
    gini_r = gini_impurity(p_right)
    return gini_l + gini_r

def draw_bootstrap(X_train, y_train):
    """Boostraping (Drawing random data samples with Replacement) the given dataset."""
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True))
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
    X_bs = X_train.iloc[bootstrap_indices].values
    y_bs = y_train[bootstrap_indices]
    X_oob = X_train.iloc[oob_indices].values
    y_oob = y_train[oob_indices]
    return X_bs, y_bs, X_oob, y_oob

def oob_score(tree, X_test, y_test):
    """OOB score is computed as the number of correctly predicted rows from the out of bag sample.
    Out-Of-Bag (OOB) error is the average error for each, zi, calculated using predictions from the 
    trees that do not contain, zi, in their respective bootstrap sample."""
    mis_label = 0
    for i in range(len(X_test)):
        pred = predict_tree(tree, X_test[i])
        if pred != y_test[i]:
            mis_label += 1
    return mis_label / len(X_test)

In [49]:
def find_split_point(X_bootstrap, y_bootstrap, max_features, impurity):
    """Given maximum no. of features at a node try to find the best split point according
    to impurity score for each threshold or possible split point."""
    feature_ls = list()
    num_features = len(X_bootstrap[0])
    
    while len(feature_ls) <= max_features:
        feature_idx = random.sample(range(num_features), 1)
        if feature_idx not in feature_ls:
            feature_ls.extend(feature_idx)
    
    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
        for split_point in X_bootstrap[:,feature_idx]:
            left_child_node = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child_node = {'X_bootstrap': [], 'y_bootstrap': []}
            
            # split children for continuous variables
            if type(split_point) in [int, float]:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value <= split_point:
                        left_child_node['X_bootstrap'].append(X_bootstrap[i])
                        left_child_node['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child_node['X_bootstrap'].append(X_bootstrap[i])
                        right_child_node['y_bootstrap'].append(y_bootstrap[i])
            # split children for categoric variables
            else:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value == split_point:
                        left_child_node['X_bootstrap'].append(X_bootstrap[i])
                        left_child_node['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child_node['X_bootstrap'].append(X_bootstrap[i])
                        right_child_node['y_bootstrap'].append(y_bootstrap[i])
            
            if impurity=='entropy':
                split_info_gain = information_gain(left_child_node['y_bootstrap'], right_child_node['y_bootstrap'])
            else:
                split_info_gain = gini_score(left_child_node['y_bootstrap'], right_child_node['y_bootstrap'])
            
            if split_info_gain > best_info_gain:
                best_info_gain = split_info_gain
                left_child_node['X_bootstrap'] = np.array(left_child_node['X_bootstrap'])
                right_child_node['X_bootstrap'] = np.array(right_child_node['X_bootstrap'])
                node = {'information_gain': split_info_gain, 
                        'left_child': left_child_node, 
                        'right_child': right_child_node, 
                        'split_point': split_point,
                        'feature_idx': feature_idx}
                
    
    return node

In [50]:
def terminal_node(node):
    """Based on max count of labels/class, assign the label to leaf node"""
    y_bootstrap = node['y_bootstrap']
    pred = max(y_bootstrap, key = y_bootstrap.count)
    return pred


def split_node(node, max_features, min_samples_split, max_depth, depth, impurity):
    """Based on Impurity score try to split the node 
    subject to constraints like max_features, min_samples_spit, max_depth and depth."""
    left_child = node['left_child']
    right_child = node['right_child']

    del(node['left_child'])
    del(node['right_child'])

    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return

    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node

    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = node['right_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features, impurity)
        split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1, impurity)
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = node['left_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features, impurity)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1, impurity)

In [51]:
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features, impurity):
    """Build decision tree given boostrapped data along with its labels 
    and the criteria to split + some extra factors."""
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features, impurity)
    split_node(root_node, max_features, min_samples_split, max_depth, 1, impurity)
    return root_node

def random_forest(X_train, y_train, n_estimators, max_features, max_depth, min_samples_split, impurity):
    """Construct Random Forest using a bunch of decision trees."""
    tree_ls = list()
    oob_ls = list()
    for i in range(n_estimators):
        X_bootstrap, y_bootstrap, X_oob, y_oob = draw_bootstrap(X_train, y_train)
        tree = build_tree(X_bootstrap, y_bootstrap, max_features, max_depth, min_samples_split, impurity)
        tree_ls.append(tree)
        oob_error = oob_score(tree, X_oob, y_oob)
        oob_ls.append(oob_error)
    print("OOB estimate for Randon Forest model: {:.2f}".format(np.mean(oob_ls)))
    return tree_ls

In [52]:
def predict_tree(tree, x_test_sample):
    """Predict the output for a single sample with a given tree."""
    feature_idx = tree['feature_idx']

    if x_test_sample[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], x_test_sample)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], x_test_sample)
        else:
            return tree['right_split']
        
def predict_all_tree(tree, X_test):
    """Return prediction for all samples in X_test"""
    if type(X_test) != np.ndarray:
        X_test = X_test.values
    return np.array([predict_tree(tree, X_test[i]) for i in range(len(X_test))])
    
def predict_rf(tree_ls, X_test):
    """Return all the predictions for test data using all trees in Random forest."""
    pred_ls = list()
    if type(X_test) != np.ndarray:
        X_test = X_test.values
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key = ensemble_preds.count)
        pred_ls.append(final_pred)
    return np.array(pred_ls)

4. Report the accuracies obtained after using the Random forest algorithm and compare it with the best accuracies obtained with the decision trees. 

**For `Gini`**

In [53]:
model_rf_gini = random_forest(X_train, y_train, n_estimators=10, max_features=3,\
                      max_depth=3, min_samples_split=2, impurity='gini')
print("============================================================================")
# Prediction for RF with Maximum voting strategy.
preds = predict_rf(model_rf_gini, X_test)
# Predict accuracy.
acc_rf = accuracy_score(y_test, preds)
acc_tree = [accuracy_score(y_test, predict_all_tree(d_tree, X_test)) for d_tree in model_rf_gini]
print("With Gini (as Impurity score):\n\nBest accuracy obtained with Decision Tree: {0:.3f}\nAccuracy obtained with Random Forest: {1:.3f}".format(max(acc_tree), acc_rf))
print("============================================================================")

OOB estimate for Randon Forest model: 0.19
With Gini (as Impurity score):

Best accuracy obtained with Decision Tree: 0.931
Accuracy obatined with Random Forets: 0.926


**For `Entropy`**

In [54]:
model_rf_entropy = random_forest(X_train, y_train, n_estimators=10, max_features=3,\
                      max_depth=3, min_samples_split=2, impurity='entropy')
print("============================================================================")
# Prediction for RF with Maximum voting strategy.
preds = predict_rf(model_rf, X_test)
# Predict accuracy.
acc_rf = accuracy_score(y_test, preds)
acc_tree = [accuracy_score(y_test, predict_all_tree(d_tree, X_test)) for d_tree in model_rf_entropy]
print("With Entropy (as Impurity score):\n\nBest accuracy obtained with Decision Tree: {0:.3f}\nAccuracy obtained with Random Forest: {1:.3f}".format(max(acc_tree), acc_rf))
print("============================================================================")

OOB estimate for Randon Forest model: 0.16
With Entropy (as Impurity score):

Best accuracy obtained with Decision Tree: 0.939
Accuracy obatined with Random Forets: 0.494


5. Submit your solution as a separate pdf in the final zip file of your submission


Compute a decision tree with the goal to predict the food review based on its smell, taste and portion size.

(a) Compute the entropy of each rule in the first stage.

(b) Show the final decision tree. Clearly draw it.

Submit a handwritten response. Clearly show all the steps.

